In [19]:
from ngsolve import *

In [20]:
# from netgen.csg import Pnt,SplineCurve2d,CSGeometry,Revolution,Sphere
import numpy as np
import netgen.meshing as ngm
from netgen.csg import *
from netgen.meshing import MeshingStep
from ngsolve.comp import IntegrationRuleSpaceSurface

In [21]:
from es_utils import pos_transformer

In [22]:
from geometry import *

In [23]:
from esfem import WillMoreMDR

In [24]:
from ngsolve.webgui import Draw

In [25]:
from netgen.occ import SplineApproximation, Pnt, Axis, Face, Wire, Segment, Revolve, OCCGeometry, Z, X, Y
#%% Construction of Initial Curved Mesh
dim = 3
order = 2
msize = 0.1
dt = Parameter(0)

Perturbed torus
\begin{equation*}
    \begin{pmatrix}
(1+0.65\cos\varphi)\cos\theta\\ (1+0.65\cos\varphi)\sin\theta\\
0.65\sin \varphi + 0.3 \sin(5 \theta)
    \end{pmatrix}
    =     \begin{pmatrix}
(1+0.65\cos\varphi)\cos\theta\\ (1+0.65\cos\varphi)\sin\theta\\
0.65\sin \varphi 
    \end{pmatrix}
    + 
        \begin{pmatrix}
0\\0\\
0.3 \sin(5 \arctan(x,y))
    \end{pmatrix}
\end{equation*}

In [26]:
def GetRotFace(CurveFunc,msize,T_min=-np.pi/2,T_max=np.pi/2,axis=Z,is_close=False,n=100):
    '''
        Generate Rotational Mesh by revolving a 2d curve (CurveFunc) around an axis (axis)
        CurveFunc: function of phi, return (x(phi), 0, z(phi)) for example
        axis: axis of rotation, default z-axis: curve on x-z plane around z-axis
        is_close: whether the curve is closed (first point = last point)
        n: number of points on the curve
        T_min, T_max: parameter range of the curve
        msize: mesh size
        return: mesh 
    '''
    pnts = [CurveFunc(phi) for phi in np.linspace(T_min,T_max,n)]
    spline = SplineApproximation(pnts, tol=1e-4)
    f = Face(Wire([spline]))
    return f

In [27]:
def BaseTorus(phi): 
    a = 0.65
    z = a*np.sin(phi)
    res = Pnt(1+a*np.cos(phi), 0, z)
    return res

In [28]:
f = GetRotFace(BaseTorus,0.08,T_min=np.pi,
                  T_max=3*np.pi,axis=Z,is_close=True)

In [29]:
f.edges[0].vertices[0].maxh = 0.05

In [30]:
from ngsolve import Mesh

In [31]:
torus = f.Revolve(Axis((0,0,0), Z), 360)
mesh = Mesh(OCCGeometry(torus).GenerateMesh(maxh=msize,
            perfstepsend=ngm.MeshingStep.MESHSURFACE,grading=0.5))

In [32]:
Draw(x,mesh,"vv")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [33]:
t = Parameter(0)
v = CF((0,0,0.1*sin(5*atan2(y,x))))

In [34]:
mesh.UnsetDeformation()
Draw(Norm(CF((v.Diff(x),v.Diff(y)))),mesh,"vv")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [35]:
DB_BGN_Obj = WillMoreMDR(mesh,T=1.5,dt=1e-5,order=1)

fes order is 1, with ndof is 2534
here


In [ ]:
sceneu = Draw(x, mesh, 'disp')
t_old = 0
tauval = 1e-2
while t_old < 3:
    t.Set(t_old+tauval)
    DB_BGN_Obj.nuold.Set(specialcf.normal(3),
                         definedon=mesh.Boundaries(".*"))
    DB_BGN_Obj.Vold.Set(InnerProduct(v,DB_BGN_Obj.nuold),definedon=mesh.Boundaries(".*"))
    DB_BGN_Obj.LapvSet() # 计算带有切向速度的 DB_BGN_Obj.vold
    DB_BGN_Obj.Disp.vec.data += BaseVector(tauval*DB_BGN_Obj.vold.vec.FV().NumPy())
    
    DB_BGN_Obj.mesh.SetDeformation(DB_BGN_Obj.Disp)
    t_old += tauval
    sceneu.Redraw()    

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

In [ ]:
import copy
tmpv = copy.deepcopy(DB_BGN_Obj.Disp.vec.FV().NumPy())

In [ ]:
import numpy as np

def param_map(theta, phi, a=1.0, b=0.65, bump=0.0, k=5):
    x = (a + b*np.cos(phi)) * np.cos(theta)
    y = (a + b*np.cos(phi)) * np.sin(theta)
    z = b*np.sin(phi) + bump * np.sin(k*theta)
    return x, y, z

def generate_grid_with_refine(n_theta=60, n_phi=30, refine_factor=2,
                              a=1.0, b=0.65, bump=0.0, k=5,
                              r_threshold=1.0, remove_duplicates=True, round_decimals=8):
    """
    返回：合并后的点云 (N,3)。策略：
      - 先生成等距粗网格 (n_theta x n_phi) ：保留全部点
      - 生成更细的网格 (refine_factor * n_theta, refine_factor * n_phi)
        并只保留满足 x^2+y^2 < r_threshold 的细网格点
      - 合并粗网格点与这些细网格点（可选去重）
    """
    # 粗网格（不含右端点以避免 seam 重复）
    thetas = np.linspace(0, 2*np.pi, n_theta, endpoint=False)
    phis   = np.linspace(-np.pi, np.pi, n_phi, endpoint=False)
    Theta, Phi = np.meshgrid(thetas, phis, indexing='ij')  # shape (n_theta, n_phi)

    x, y, z = param_map(Theta, Phi, a=a, b=b, bump=bump, k=k)
    coords_coarse = np.column_stack([x.ravel(), y.ravel(), z.ravel()])

    # 细网格
    n_theta_f = n_theta * refine_factor
    n_phi_f   = n_phi * refine_factor
    thetas_f = np.linspace(0, 2*np.pi, n_theta_f, endpoint=False)
    phis_f   = np.linspace(-np.pi/10, np.pi/10, n_phi_f, endpoint=False)
    phis_f2   = np.linspace(np.pi-np.pi/10, np.pi+np.pi/10, n_phi_f, endpoint=False)
    Theta_f, Phi_f = np.meshgrid(thetas_f, np.concatenate([phis_f,phis_f2]), indexing='ij')

    x_f, y_f, z_f = param_map(Theta_f, Phi_f, a=a, b=b, bump=bump, k=k)
    mask_f = (x_f**2 + y_f**2) < r_threshold
    coords_fine_masked = np.column_stack([x_f[mask_f], y_f[mask_f], z_f[mask_f]])

    # 合并：保留所有粗网格点 + 加密区域的细点
    coords_all = np.vstack([coords_coarse, coords_fine_masked])

    if remove_duplicates:
        # 去重：按网格坐标四舍五入后用 np.unique 去重（保守且简单）
        coords_round = np.round(coords_all, decimals=round_decimals)
        # np.unique 对行去重
        coords_unique = np.unique(coords_round, axis=0)
        return coords_unique

    return coords_all

pts = generate_grid_with_refine(n_theta=800, n_phi=400, refine_factor=4,
                                a=1.0, b=0.65, bump=0.3, k=5,
                                r_threshold=1.0, remove_duplicates=True)

In [ ]:
Vertices_Coords = DB_BGN_Obj.Get_Vertices_Coords()

In [24]:
from scipy.spatial import cKDTree
# 构建 kd-tree
tree = cKDTree(pts)
distances, indices = tree.query(Vertices_Coords, k=1) 

disp = pts[indices,:] - Vertices_Coords

In [25]:
DB_BGN_Obj.Disp.vec.data = BaseVector(tmpv + disp.flatten("F"))

In [26]:
DB_BGN_Obj.mesh.SetDeformation(DB_BGN_Obj.Disp)

In [27]:
Draw(x, DB_BGN_Obj.mesh, 'disp')

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.23…

BaseWebGuiScene

In [28]:
order = 1
mesh_name = "../../../data/order_{}_wave.vol".format(order)
DB_BGN_Obj.mesh.ngmesh.Save (mesh_name)
print(mesh_name)

np.savez("../../../data/order_{}_deformation_wave.npz".format(order), 
         deform=DB_BGN_Obj.Disp.vec.FV().NumPy())

../../../data/order_1_wave.vol


### 如何得到二次的deformation